In [ ]:
from pathlib import Path

import numpy as np

import FewShotCVRP.ea as ea
import FewShotCVRP.ea_parallel as ea_parallel
from FewShotCVRP.dataset.theta_control_loader import DescriptorRepository
from FewShotCVRP.examples.params_search import simulation


In [3]:
descriptor_path = Path("../../dataset/trained_models/per_instance/descriptor.json")
repo = DescriptorRepository(descriptor_path)

instance = next(inst for inst in repo.instances if inst.name == "X-n209-k16")
cvrp = simulation.get_cvrp_instance("X-n209-k16.xml")

cfg = repo.ea_config
lambda_ = int(cfg["ea_lambda_"])
generations_number = int(cfg["ea_generations_number"])
max_evals = generations_number * lambda_
seed = 2024


In [ ]:
wrapper_controller = repo.create_theta_controller(instance)
baseline_controller = repo.create_theta_controller(instance)

theta_schedule_window = np.asarray(baseline_controller.theta_schedule_window(), dtype=float)
theta_control_fun = baseline_controller._theta_function


In [ ]:
res_wrapper = ea_parallel.one_plus_lambda_ea_parallel_nn_control(
    cvrp,
    wrapper_controller,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation="2opt",
    numproc=1,
    verbose=False,
)

res_original = ea.one_plus_lambda_ea_with_theta_control(
    cvrp,
    theta_control_fun=theta_control_fun,
    window=len(theta_schedule_window),
    theta_schedule_window=theta_schedule_window,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation=ea.mutate_shift_2opt_fast_fast,
    verbose=False,
)


In [5]:
comparison = {
    "best_fitness_equal": np.isclose(res_wrapper["best_fitness"], res_original["best_fitness"]),
    "evals_equal": res_wrapper["evals"] == res_original["evals"],
    "gens_equal": res_wrapper["gens"] == res_original["gens"],
    "history_equal": res_wrapper["history"] == res_original["history"],
}
comparison


{'best_fitness_equal': np.True_,
 'evals_equal': True,
 'gens_equal': True,
 'history_equal': True}

In [45]:
from importlib import reload

reload(ea_parallel)

res_wrapper = ea_parallel.one_plus_lambda_ea_parallel_nn_control(
    cvrp,
    wrapper_controller,
    lambda_=lambda_,
    max_evals=max_evals,
    generations_number=generations_number,
    seed=seed,
    mutation="2opt",
    numproc=50,
    verbose=False,
)
res_wrapper["best_fitness"]

52417.0